In [1]:
%load_ext ipython_sparql_pandas
from SPARQLWrapper import SPARQLWrapper, JSON
sparql = SPARQLWrapper('http://DESKTOP-CELL0BF:7200/repositories/20qsmall')
sparql.setReturnFormat(JSON)

In [2]:
#from the query results extract a list that contains the counts after splitting on certain attributes
import re
PosAttr=['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .', '?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.']
NegAttr=[]
AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']

def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i]= 1/len(popEntities)
    return Distribution


def extractCountAndAttr(result):
    number = []
    listOfObj = []
    listOfPred= []
    for attr in result["results"]["bindings"]:
        listOfObj.append(attr['o'])
        listOfPred.append(attr['p'])
    #print(listOfObj[0])
    return(listOfObj,listOfPred)

def generateQuestion(listOfPred, listOfObj, index):
    flag = True
    if ('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>') in AttrHistory:
        return generateQuestion(listOfPred, listOfObj, index+1)
        
    else:
        AttrHistory.append('<' + str(listOfPred[index]['value']) +'> <'+str(listOfObj[index]['value'])+ '>')
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    try:
        int(listOfObj[index]['value'])
    except ValueError:
           flag = False
            
    if userAnswer.startswith('y'):
        PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.') 
        """if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
              """ 
        
    elif userAnswer.startswith('n'):
        NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}') 
        """if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
              """
        
    else:
        return generateQuestion(listOfPred, listOfObj, index+1)
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


  
def updateQuery(left, PosFilters, NegFilters):
    FilterQuestion =  (f"""
            select (count(*) as ?count) ?p ?o  where 
            {{
            {PosFilters}
             ?s ?p ?o .
            {NegFilters}
            }} 
            group by ?p ?o 
            ORDER BY ABS( {left/2} - ?count )
            """) 
        
    return FilterQuestion

In [3]:
def numberleft(PosFilters,NegFilters):
    query =  f"""
            select (count(?s) as ?count) where 
            {{
            {PosFilters}            
            {NegFilters} 
          }}       
            group by ?p ?o 
            ORDER BY DESC(?count )            
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        left= (r["count"]["value"])
        return(int(left))
    
def popentities(PosFilters,NegFilters):
    query =  f"""
            select ?s where 
            {{
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
            ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
            {PosFilters}
            {NegFilters} 
            }}
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    results = []
    for r in qres["results"]["bindings"]:
        results.append(r["s"]["value"])
    return results

In [4]:
# resets are variables
def game_reset():
    PosAttr= ['?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .', '?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.']
    NegAttr= ['']
    AttrHistory = ['<http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity>']
    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    left = numberleft(PosFilters,NegFilters)
    FilterQuestion = updateQuery(left, PosFilters, NegFilters)
    sparql.setQuery(FilterQuestion)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert()
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    return (left, qres, PosAttr, NegAttr, AttrHistory)

In [10]:
def prepareProps(popEntities):
    Distribution = {}
    for i in popEntities:
        Distribution[i]= 1/len(popEntities)
    return Distribution
    
print(prepareProps(["Oma", "Opa"]))

def findPosAndNeg(listOfObj, listOfPred, index):
    Filter = '?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.'
    negFilter='FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}'
    EntitiesPositive = popentities(Filter, "")
    EntitiesNegative = popentities("", negFilter)
    return (EntitiesPositive, EntitiesNegative)
    

def generateQuestion(listOfObj, listOfPred, index, Distribution):

    AttrHistory.append('<' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>')
    
    userAnswer = input(f'Does the thing you are looking for have the attribute: {listOfPred[index]} {listOfObj[index]}?' )
    Yes, No = findPosAndNeg(listOfObj, listOfPred, index)
    if userAnswer.startswith('y'):
        for y in Yes:
            Distribution[y] = Distribution[y] * 2
        for n in No:
            Distribution[n] = Distribution[n] / 2
            
    elif userAnswer.startswith('n'):
        for y in Yes:
            Distribution[y] = Distribution[y] / 2
        for n in No:
            Distribution[n] = Distribution[n] * 2
    
    return Distribution
                       
def findSplits():
    possibleObj=[]
    possiblePred =[]
    count = []
    query =  f"""
            select (count(?s) as ?count)   ?o ?p
            {{               
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
        ?s ?p ?o.
            }}
        group by  ?o ?p
        ORDER BY desc ( ?count )
            """ 
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"]:
        possibleObj.append(r["o"]["value"])
        possiblePred.append(r["p"]["value"])
        count.append(r["count"]["value"])
   
    return (possibleObj, possiblePred, count)

def determineBestSplit(possibleObj, possiblePred, count):
    bestSplit=-1
    indexBestSplit = -1
    for i in range(125): #afterwards first numeral appears which causes issues
        score=0
        #Fix The nummerical thing here::: (skip over i if nummer)
        if str('<' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>') in AttrHistory:
            continue
        else:
            Filter = '?s <' + str(possiblePred[i]) +'> <'+str(possibleObj[i])+ '>.'
            EntitiesWithInfo = popentities(Filter, "")
            for ent in EntitiesWithInfo:
                score += int(Distribution.get(ent))
            if score > bestSplit:
                bestSplit = score
                indexBestSplit = i
    return(indexBestSplit)

        
                             
"""preparePops
enter loop
find splits, determine best split, generate question (update probabilities)"""
                             
                             

{'Oma': 0.5, 'Opa': 0.5}


'preparePops\nenter loop\nfind splits, determine best split, generate question (update probabilities)'

In [13]:
import random
from random import * 
i=0
left, qres, PosAttr, NegAttr, AttrHistory = game_reset()
outofguess=[]
popularEntities =  popentities("\n".join(PosAttr),"")
Distribution = prepareProps(popularEntities)
while i<5:
    listOfObj ,listOfPred = extractCountAndAttr(qres)
    if left <=1:
        break
    print('Question: %s' %(i+1))
    
    possibleObj, possiblePred, count = findSplits()
    indexSplit=determineBestSplit(possibleObj, possiblePred, count)
    Distribution = generateQuestion(possibleObj, possiblePred, indexSplit, Distribution)
  
    i+=1
    


if i == 5:
    highProb=-1
    for j in Distribution.values():
        print(j)
        if j > highProb:
            highProb=j
    answer = list(Distribution.keys())[list(Distribution.values()).index(j)]
    print("The Answer is: ", answer, "With a Probability of: ", highProb)
else:
    print('Final Question:')
    outofguess=popentities(PosFilters, NegFilters)
    guess = random.choice(outofguess)
    print(guess)
    answer = (input('Is this correct?').startswith('y'))
    if answer is True:
        print('Win')
    else:
        print('game over')

Question: 1


Does the thing you are looking for have the attribute: http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://yago-knowledge.org/resource/Human? no


Question: 2


Does the thing you are looking for have the attribute: http://schema.org/knowsLanguage http://yago-knowledge.org/resource/English_language? y


Question: 3


...

Question: 4


Does the thing you are looking for have the attribute: http://schema.org/hasOccupation http://yago-knowledge.org/resource/Actor? y


Question: 5


Does the thing you are looking for have the attribute: http://schema.org/hasOccupation http://yago-knowledge.org/resource/Film_producer? n


0.014925373134328358
0.05970149253731343
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.0037313432835820895
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.0037313432835820895
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.05970149253731343
0.0037313432835820895
0.014925373134328358
0.0037313432835820895
0.05970149253731343
0.014925373134328358
0.05970149253731343
0.0037313432835820895
0.014925373134328358
0.014925373134328358
0.0037313432835820895
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.0037313432835820895
0.0037313432835820895
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.0037313432835820895
0.014925373134328358
0.05970149253731343
0.014925373134328358
0.014925373134328358
0.014925373134328358
0.05970149253731343
0.01492537

In [12]:
def generateBotQuestion(listOfPred, listOfObj, index, subject):
    flag = True
    try:
        int(listOfObj[index]['value'][0])
    except ValueError:
           flag = False
    print(f'Does the thing you are looking for have the attribute: {listOfPred[index]["value"].split("/")[-1]} {listOfObj[index]["value"].split("/")[-1]}?' )
    
    if flag is True:
        datatype = listOfObj[index]["datatype"].split("#")[-1]
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '"'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype)
    else:
        answer = findAnswer(subject, '<' + str(listOfPred[index]["value"]) +'>', '<'+str(listOfObj[index]["value"])+ '>')
    print(answer)
    
    if answer:
        #PosAttr.append('?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>.')    
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'.') 
        else:
            PosAttr.append('?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>.')
        
    elif not answer:
        #NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]) +'> <'+str(listOfObj[index])+ '>. }}')   
        if flag is True:
            datatype = listOfObj[index]["datatype"].split("#")[-1]
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> "'+str(listOfObj[index]["value"])+'"^^xsd:'+datatype+'. }}')
        else:
            NegAttr.append('FILTER NOT EXISTS {{ \n ?s <' + str(listOfPred[index]["value"]) +'> <'+str(listOfObj[index]["value"])+ '>. }}')   

    NegFilters = "\n".join(NegAttr)
    PosFilters = "\n".join(PosAttr)
    return (PosFilters, NegFilters)


def pickSubject():
    ind=randint(0,134)
    query = f"""
    select distinct ?s
    where {{
        ?s ?p ?o .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://example.com/popularEntity> .
        ?s <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://yago-knowledge.org/resource/Human>.
    }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    for r in qres["results"]["bindings"][ind:ind+1]:
        return("<"+r["s"]["value"]+">")

def findAnswer(subject,p,o):
    query = f"""
        SELECT ?s ?p ?o
        WHERE {{
        ?s ?p ?o .
            FILTER(?s = {subject})
            FILTER(?p = {p})
            FILTER(?o = {o})
            }}
            """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    qres = sparql.query().convert() 
    res = qres["results"]["bindings"]
    return (len(res))>0

In [13]:
#without printing results
from random import randint
import random

wincount = 0
gamesplayed = 0
totalgames = 25

while gamesplayed < totalgames:
    left,qres,PosAttr,NegAttr,AttrHistory = game_reset()
    # Resets all variables
    i=0
    outofguess = []

    # pick subject
    subject = pickSubject()
    print("\n" , subject)

    while i < 20:

        # list of Pred and Obj for questions
        listOfObj ,listOfPred = extractCountAndAttr(qres)

        # if only 1 popular entity is left break
        if left<=1:
            break

        # ask question and add filters to query
        print('Question: %s' %(i+1))
        PosFilters, NegFilters = generateBotQuestion(listOfPred, listOfObj, 0, subject)

        # number of popular entities left
        left = numberleft(PosFilters, NegFilters)

        # run query
        query = updateQuery(left/2,PosFilters, NegFilters)
        sparql.setQuery(query)
        qres = sparql.queryAndConvert()

        i+=1

    # Prints answer
    if i != 20:
        for attr in popentities(PosFilters, NegFilters):
            for x in [attr['s']['value']][0:10]:
                answer = (x.split("/")[-1])
        print("Your Answer is: " + answer)
        if str(answer.split("/")[-1] + ">") == str(subject.split("/")[-1]):
            wincount += 1
            gamesplayed += 1
            print("Win!")
        else:
            gamesplayed += 1
            print("Lose...")

    # ran out of questions
    else:
        print('Final Question:')
        for j in popentities(PosFilters, NegFilters):
                outofguess.append(f'{j["s"]["value"].split("/")[-1]}')
        guess = random.choice(outofguess)
        print("Is it " + guess + "?")
        if str(subject.split("/")[-1]) == str(guess + ">"):
            wincount += 1
            gamesplayed += 1
            print('Win!')
        else:
            gamesplayed += 1
            print('Lose...')
            
print("\nTournament ended! I have guessed " + str(wincount) + " out of " + str(gamesplayed) + " things correctly.")


 <http://yago-knowledge.org/resource/Albert_Einstein>
Question: 1
Does the thing you are looking for have the attribute: hasOccupation Actor?
False
Question: 2
Does the thing you are looking for have the attribute: memberOf Democratic_Party_(United_States)?
False
Question: 3
Does the thing you are looking for have the attribute: hasOccupation Politician?
False
Question: 4
Does the thing you are looking for have the attribute: hasOccupation Writer?
True
Question: 5
Does the thing you are looking for have the attribute: memberOf American_Philosophical_Society?
True
Question: 6
Does the thing you are looking for have the attribute: image Albert%20Einstein%20Head.jpg?
True
Your Answer is: Albert_Einstein
Win!

 <http://yago-knowledge.org/resource/Marilyn_Manson>
Question: 1
Does the thing you are looking for have the attribute: hasOccupation Actor?
True
Question: 2
Does the thing you are looking for have the attribute: hasOccupation Stage_actor?
False
Question: 3
Does the thing you are lo

KeyboardInterrupt: 

In [81]:
try:
    int('1946-07-06')
except ValueError:
    print("baguette")

baguette


In [88]:
Flag= True
try:
    int(1946-07-06)
except ValueError:
       Flag = False
            
if Flag is False:
    print("fuck")

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-88-e56ed9eb6169>, line 3)